In [27]:
"""
Several parenclitic functions
"""

import numpy as np
import pandas as pd
import networkx as nx
import statsmodels.api as sma
import warnings
warnings.filterwarnings("ignore")

from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity

############### Probably needed functions

In [28]:
"""
Загрузка ideal_sphere_10_65_65_20_20.csv

"""

data = pd.read_csv('/home/guest/Документы/Parenclitic/spheres/ideal_sphere_10_65_65_20_20.csv')

data = data.drop('Unnamed: 0', axis=1)

nodes_lst = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']

In [111]:
def zanin_preprocess_scaler(X_train, X_test, y_train, nodes_lst, training_index=None):

    sc = StandardScaler()

    sc = sc.fit(X_train.loc[training_index,:][nodes_lst])
    X_train[nodes_lst] = abs(sc.transform(X_train[nodes_lst]))
    
    #if X_test.shape[0] > 0:
    try: 
        if X_test.shape:
            X_test[nodes_lst] = abs(sc.transform(X_test[nodes_lst]))
    except:
        pass

    return(X_train, X_test)


def zanin_model_fitter(X_train, y_train, nodes_lst, training_index=None):

    X_train_regr = X_train.loc[training_index,:]
    feature_lst = []
    model_lst = []

    for i in list(combinations(nodes_lst, r = 2)): 

        a = list(i)
        feature_lst = feature_lst + [a[0] + '-' + a[1]]

        X = sma.add_constant(X_train_regr[a[1]])
        y = X_train_regr[a[0]]
        model_ols = sma.OLS(y, X).fit()

        model_lst = model_lst + [model_ols]

    """         if transform:
            for df in [X_train, X_test]:
                df[a[0] + '-' + a[1]] = model_ols.predict(
                            sma.add_constant(df[a[1]])) - df[a[0]] """
        
        
    return(feature_lst, model_lst)


def zanin_fit(df, model_lst, nodes_lst, drop_nodes=True):

    for i, mod in zip(list(combinations(nodes_lst, r = 2)), model_lst): 

        a = list(i)

        df[a[0] + '-' + a[1]] = mod.predict(
                            sma.add_constant(df[a[1]])) - df[a[0]]

    if drop_nodes:
        df = df.drop(nodes_lst, axis=1)

    return(df)


def graph_basket(df_matrix, nodes_lst, tuples_lst, feature_lst, id):
    """
    df_matrix - таблица с фичами, связями и весами для графов, уникальным номером пациента
    nodes_lst - список с иемнами узлов
    tuples_lst - список с кортежами, содержащий связи между 2-мя узлами (отражает будущее ребро)
    feature_lst - список с именами ребер, соответствующий элементам списка с кортежами tuples_lst
    id - колонка с уникальным номером пациента
    """
    G_list = []
    uins_lst = []

    for elems in df_matrix[id]:

        # создание графа
        G = nx.Graph()
        # Создание узлов
        G.add_nodes_from(nodes_lst)
        # Создание каждого ребра из tuples_lst
        for e, w in zip(tuples_lst, feature_lst):
            G.add_edge(e[0], e[1], weight = float(df_matrix[df_matrix[id]==elems][w]))

        # Заполнение списков с пациентами и с графами:
        uins_lst = uins_lst + [elems]
        G_list = G_list + [G]

    return(uins_lst, G_list)


def graph_multi_centralities(uins_lst, G_list):

    btwnns = []
    clsns = []
    edge_btwnns = []
    pgrnk = []
    eign = []
    auth = []
    strength = []

    future_data = {}

    for gg in G_list:
        clsns = clsns + [list(nx.closeness_centrality(gg, distance='weight', wf_improved=False).values())]
        btwnns = btwnns + [list(nx.betweenness_centrality(gg, weight='weight', normalized=False).values())]
        edge_btwnns = edge_btwnns + [list(nx.edge_betweenness_centrality(gg, weight='weight', normalized=False).values())]
        pgrnk = pgrnk + [list(nx.pagerank(gg, weight='weight').values())]
        eign = eign + [list(nx.eigenvector_centrality(gg, weight='weight').values())]
        auth = auth + [list(nx.hits(gg)[1].values())]
        strength = strength + [list(dict(gg.degree(weight='weight')).values())]

    for cent, name in zip([btwnns, clsns, edge_btwnns, pgrnk, eign, auth, strength], \
        ['btwnns', 'clsns', 'edge_btwnns', 'pgrnk', 'eign', 'auth', 'strength']):

        future_data[name + '_mean']=np.array(cent).mean(axis=1)
        future_data[name + '_sd']=np.array(cent).std(axis=1)
        future_data[name + '_min']=np.array(cent).min(axis=1)
        future_data[name + '_max']=np.array(cent).max(axis=1)

    future_data['id'] = uins_lst

    return(pd.DataFrame(future_data))


def zanin_X_train_fit(X_train, y_train, nodes_lst, training_index, pre_scale=False):

    if pre_scale:
        X_train, _ = zanin_preprocess_scaler(
                        X_train, 
                        None, 
                        y_train, 
                        nodes_lst, 
                        training_index)

    feature_lst, model_lst = zanin_model_fitter(
                                X_train, 
                                y_train, 
                                nodes_lst,
                                training_index)

    tuples_lst = [tuple(i.split('-')) for i in feature_lst]

    fitter = zanin_fit(X_train, model_lst, nodes_lst, drop_nodes=False)

    return(fitter, feature_lst, model_lst, tuples_lst)
    

def zanin_transform(
        fitter, 
        X_test, 
        nodes_lst, 
        training_index, 
        feature_lst, 
        model_lst, 
        tuples_lst,
        pre_scale=False):

    if pre_scale:
        _, X_test = zanin_preprocess_scaler(
            X_train, 
            X_test, 
            y_train, 
            nodes_lst,
            training_index)

    X_test = zanin_fit(X_test, model_lst, nodes_lst)


    _, X_test = zanin_preprocess_scaler(
                    X_train[feature_lst], 
                    X_test, 
                    y_train, 
                    feature_lst,
                    training_index)

    X_test['id'] = X_test.index

    uins_lst, G_list = graph_basket(X_test, nodes_lst, tuples_lst, feature_lst, id='id')

    graph_chars = graph_multi_centralities(uins_lst, G_list)

    return graph_chars



In [113]:
X_train = data[data['label'] == 'TRAIN'].set_index('id').drop(['label', 'score'], axis=1)
X_test = data[data['label'] == 'TEST'].set_index('id').drop(['label', 'score'], axis=1)
y_train = data[data['label'] == 'TRAIN'].set_index('id')['score']
y_test = data[data['label'] == 'TEST'].set_index('id')['score']

fitter, feature_lst, model_lst, tuples_lst = zanin_X_train_fit(X_train, y_train, nodes_lst, y_train[y_train == 0].index)

zanin_transform(fitter, X_train, nodes_lst, y_train[y_train == 0].index, feature_lst, model_lst, tuples_lst)

#zanin_transform(X_train, X_train, nodes_lst, y_train[y_train == 0].index, feature_lst, model_lst, tuples_lst)



btwnns_mean  btwnns_sd  btwnns_min  btwnns_max  clsns_mean  clsns_sd  \
0            3.1   8.055433         0.0        27.0    5.353308  1.661045   
1            2.0   5.079370         0.0        17.0    0.784547  0.221034   
2            1.2   2.749545         0.0         9.0    2.333186  0.415002   
3            4.6   7.337575         0.0        19.0   11.732242  3.568517   
4            1.7   5.100000         0.0        17.0    1.186006  0.248610   
..           ...        ...         ...         ...         ...       ...   
125          3.5   3.471311         0.0        10.0    1.121059  0.270569   
126          5.9   8.190849         0.0        27.0    1.259848  0.395857   
127          2.8   6.645299         0.0        22.0    0.903870  0.217413   
128          5.0   8.532292         0.0        23.0    1.323311  0.410003   
129          3.3   9.900000         0.0        33.0    0.619458  0.181662   

     clsns_min  clsns_max  edge_btwnns_mean  edge_btwnns_sd  ...  eign_max  \
0     1.700979   7.637381          1.688889        3.046471  ...  0.387740   
1     0.458668   1.079997          1.444444        2.226662  ...  0.367701   
2     1.859558   3.147155          1.266667        1.372751  ...  0.380317   
3     7.030424  16.687854          2.022222        3.838338  ...  0.494200   
4     0.821814   1.801008          1.377778        1.889020  ...  0.428171   
..         ...        ...               ...             ...  ...       ...   
125   0.630602   1.478735          1.777778        2.882043  ...  0.504868   
126   0.286062   1.656059          2.311111        4.852593  ...  0.448667   
127   0.527403   1.292933          1.622222        2.618642  ...  0.469759   
128   0.263082   1.714257          2.111111        4.342156  ...  0.421466   
129   0.390950   1.007520          1.733333        3.335999  ...  0.414003   

     auth_mean   auth_sd  auth_min  auth_max  strength_mean  strength_sd  \
0          0.1  0.025932  0.033418  0.126670       3.896040     1.147732   
1          0.1  0.017228  0.058611  0.117990      15.823977     3.260701   
2          0.1  0.018023  0.064262  0.122205       4.480778     0.853702   
3          0.1  0.054349  0.014956  0.177870       4.215012     2.344383   
4          0.1  0.023176  0.054060  0.138988       9.438023     2.452776   
..         ...       ...       ...       ...            ...          ...   
125        0.1  0.042565  0.049266  0.173515      19.340615     8.777742   
126        0.1  0.032509  0.039887  0.149190      18.379391     7.458977   
127        0.1  0.034597  0.036946  0.157190      18.408728     6.817311   
128        0.1  0.033871  0.035876  0.140717      22.319104     8.929074   
129        0.1  0.028772  0.028737  0.136231      32.453006    10.571589   

     strength_min  strength_max   id  
0        1.205990      5.470496    1  
1        8.676263     19.663387    2  
2        2.859725      5.579207    3  
3        0.651937      7.863959    4  
4        4.997202     13.895770    5  
..            ...           ...  ...  
125     11.190693     39.224949  126  
126      6.511967     32.894366  127  
127      7.155681     31.405787  128  
128      7.089954     37.743275  129  
129      8.932822     47.720039  130  

[130 rows x 29 columns]

In [29]:
X_train = data[data['label'] == 'TRAIN'].set_index('id').drop(['label', 'score'], axis=1)
X_test = data[data['label'] == 'TEST'].set_index('id').drop(['label', 'score'], axis=1)
y_train = data[data['label'] == 'TRAIN'].set_index('id')['score']
y_test = data[data['label'] == 'TEST'].set_index('id')['score']

X_train, X_test = zanin_preprocess_scaler(
    X_train, 
    X_test, 
    y_train, 
    nodes_lst,
    training_index=y_train[y_train == 0].index)

feature_lst, model_lst = zanin_model_fitter(
                            X_train, 
                            y_train, 
                            nodes_lst,
                            training_index=y_train[y_train == 0].index)

X_train = zanin_fit(X_train, model_lst, nodes_lst)
X_test = zanin_fit(X_test, model_lst, nodes_lst)

tuples_lst = [tuple(i.split('-')) for i in feature_lst]

X_train, X_test = zanin_preprocess_scaler(
                    X_train, 
                    X_test, 
                    y_train, 
                    feature_lst,
                    training_index=y_train[y_train == 0].index)

X_train = X_train.reset_index()
X_test = X_test.reset_index()

uins_lst_1, G_list_1 = graph_basket(X_train, nodes_lst, tuples_lst, feature_lst, id='id')
uins_lst_2, G_list_2 = graph_basket(X_test, nodes_lst, tuples_lst, feature_lst, id='id')

#G_list_2
graph_multi_centralities(uins_lst_1 + uins_lst_2, G_list_1 + G_list_2)



btwnns_mean  btwnns_sd  btwnns_min  btwnns_max  clsns_mean  clsns_sd  \
0            5.8   5.775812         0.0        15.0    8.450398  2.853822   
1            2.0   2.932576         0.0         8.0    0.668562  0.110780   
2            7.7   7.975588         0.0        18.0    4.660961  1.579769   
3            5.3   6.827152         0.0        17.0    1.493521  0.424780   
4            6.9   9.170060         0.0        23.0    3.538548  1.153106   
..           ...        ...         ...         ...         ...       ...   
165          3.8   5.775812         0.0        18.0    0.372153  0.113728   
166          4.3   6.116371         0.0        17.0    0.934485  0.253778   
167          6.8   5.741080         0.0        18.0    0.888064  0.221285   
168          3.9   6.073714         0.0        16.0    1.324327  0.449931   
169          3.5   7.864477         0.0        26.0    0.658381  0.144602   

     clsns_min  clsns_max  edge_btwnns_mean  edge_btwnns_sd  ...  eign_max  \
0     3.755050  11.796939          2.288889        4.534423  ...  0.556567   
1     0.523602   0.806812          1.444444        1.995056  ...  0.364323   
2     2.448154   6.323788          2.711111        5.848541  ...  0.447657   
3     0.865967   2.030963          2.177778        4.468601  ...  0.410967   
4     1.676554   4.821119          2.533333        5.721694  ...  0.466190   
..         ...        ...               ...             ...  ...       ...   
165   0.154023   0.472737          1.844444        3.217698  ...  0.457262   
166   0.492852   1.257783          1.955556        3.533683  ...  0.456826   
167   0.579644   1.197017          2.511111        4.847094  ...  0.512755   
168   0.599159   1.765775          1.866667        3.249615  ...  0.452073   
169   0.470196   0.955670          1.777778        3.168518  ...  0.417915   

     auth_mean   auth_sd  auth_min  auth_max  strength_mean  strength_sd  \
0          0.1  0.039736  0.042069  0.189388       3.644253     2.070866   
1          0.1  0.009586  0.085538  0.115737      14.953195     2.155586   
2          0.1  0.022430  0.081182  0.145079       4.372021     1.254529   
3          0.1  0.019178  0.073099  0.132328       8.209424     2.009835   
4          0.1  0.034727  0.042438  0.156059       6.750627     2.734265   
..         ...       ...       ...       ...            ...          ...   
165        0.1  0.025554  0.073755  0.149245      34.431828    13.843294   
166        0.1  0.028237  0.041780  0.150110      26.480040     8.424002   
167        0.1  0.031025  0.056091  0.169771      29.095356    11.549216   
168        0.1  0.038015  0.030877  0.152940      20.971975     7.900084   
169        0.1  0.027435  0.035053  0.137040      28.537672     8.073577   

     strength_min  strength_max   id  
0        1.687299      8.955861    1  
1       12.072328     18.224272    2  
2        3.240995      6.948760    3  
3        5.464842     11.731258    4  
4        2.970286     11.684358    5  
..            ...           ...  ...  
165     21.106889     61.488412  166  
166     11.597491     42.767094  167  
167     17.169436     57.986568  168  
168      6.963244     33.964724  169  
169      9.637026     39.825550  170  

[170 rows x 29 columns]

In [1]:
X_train = data[data['label'] == 'TRAIN'].set_index('id').drop(['label', 'score'], axis=1)
X_test = data[data['label'] == 'TEST'].set_index('id').drop(['label', 'score'], axis=1)
y_train = data[data['label'] == 'TRAIN'].set_index('id')['score']
y_test = data[data['label'] == 'TEST'].set_index('id')['score']

X_train, X_test = zanin_preprocess_scaler(
    X_train, 
    X_test, 
    y_train, 
    nodes_lst,
    training_index=y_train[y_train == 0].index)

feature_lst, model_lst = zanin_model_fitter(
                            X_train, 
                            y_train, 
                            nodes_lst,
                            training_index=y_train[y_train == 0].index)

X_train = zanin_fit(X_train, model_lst, nodes_lst)
X_test = zanin_fit(X_test, model_lst, nodes_lst)

tuples_lst = [tuple(i.split('-')) for i in feature_lst]

X_train, X_test = zanin_preprocess_scaler(
                    X_train, 
                    X_test, 
                    y_train, 
                    feature_lst,
                    training_index=y_train[y_train == 0].index)

X_train = X_train.reset_index()
X_test = X_test.reset_index()

uins_lst_1, G_list_1 = graph_basket(X_train, nodes_lst, tuples_lst, feature_lst, id='id')
uins_lst_2, G_list_2 = graph_basket(X_test, nodes_lst, tuples_lst, feature_lst, id='id')

#G_list_2
graph_multi_centralities(uins_lst_1 + uins_lst_2, G_list_1 + G_list_2)


NameError: name 'data' is not defined